# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
weather_df = pd.read_csv("../output/cities.csv")
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Price,39.5994,-110.8107,42.80,42,90,9.22
1,1,Tasiilaq,65.6145,-37.6368,30.20,86,75,33.38
2,2,Puerto Ayora,-0.7393,-90.3518,80.85,78,40,3.44
3,3,Bethel,41.3712,-73.4140,49.17,96,100,1.99
4,4,Hilo,19.7297,-155.0900,56.39,88,90,6.91
...,...,...,...,...,...,...,...,...
569,569,Broken Hill,-31.9500,141.4333,55.40,54,99,8.34
570,570,Klyuchi,52.2667,79.1667,54.72,62,80,4.59
571,571,Samandağ,36.0842,35.9771,71.60,43,0,14.97
572,572,Burnie,-41.0667,145.9167,43.47,89,18,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Configure gamps with API key
gmaps.configure(api_key=g_key)

#Use Lat and Lng as location
location = weather_df[["Lat","Lng"]].astype(float)

#Use humidity as weight
humidity = weather_df["Humidity"].astype(float)

In [16]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights = humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#create a new DF matching given weather critera

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
